In [2]:
#read mat
import pandas as pd
from scipy.io import loadmat

In [12]:
from scipy.io import loadmat

# 讀入你的 .mat 檔案
data = loadmat('snp452-data.mat')

# 看一下有哪些變數
print(data.keys())
useful_keys = [k for k in data.keys() if not k.startswith('__')]
print(useful_keys)

dict_keys(['__header__', '__version__', '__globals__', 'stock', 'X'])
['stock', 'X']


In [14]:
import pandas as pd
import numpy as np

data = loadmat('snp452-data.mat')
useful_keys = [k for k in data.keys() if not k.startswith('__')]

dfs = {}

for key in useful_keys:
    value = data[key]
    
    # 這裡要小心，有些是 list of dict，有些是純數組
    if isinstance(value, np.ndarray):
        if value.dtype.names:  # 這是結構化數組 (Structured array)
            df = pd.DataFrame.from_records(value)
        else:
            df = pd.DataFrame(value)
    else:
        df = pd.DataFrame(value)

    dfs[key] = df

# dfs是一個字典，裡面存了所有表格
print(dfs.keys())  # 查看有哪些DataFrame

dict_keys(['stock', 'X'])


In [20]:
dfs['X']

,0,1,2,3,4,5,6,7,8,9,...,442,443,444,445,446,447,448,449,450,451
0,126.75,30.51,40.05,21.75,25.60,7.01,3.24,42.04,31.59,19.14,...,25.65,13.50,11.35,8.16,22.03,80.23,17.60,25.12,40.30,40.64
1,126.27,30.69,40.44,21.34,26.30,6.94,3.45,42.33,31.60,19.05,...,25.59,13.40,11.54,8.28,22.94,80.70,18.10,24.55,40.40,40.27
2,127.19,31.97,40.70,22.11,27.51,7.16,3.45,42.75,32.00,19.96,...,26.48,13.29,12.25,8.44,24.24,82.97,18.94,24.99,40.66,41.39
3,127.07,31.78,38.87,22.07,28.51,7.17,3.37,42.18,31.73,19.78,...,26.05,13.31,11.80,8.31,25.45,82.51,19.15,25.15,40.26,41.59
4,125.70,31.17,39.85,22.40,27.13,6.69,3.35,42.23,31.80,18.82,...,26.11,13.02,12.60,8.27,24.71,81.47,18.75,25.27,40.00,42.03
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1253,86.26,63.15,58.35,83.85,42.84,7.77,22.09,58.40,62.52,36.98,...,49.67,113.28,22.99,16.38,22.44,52.58,24.05,38.66,67.42,48.83
1254,86.07,62.86,57.88,82.54,43.12,7.77,22.13,58.82,62.77,37.47,...,49.13,117.67,22.89,16.49,22.50,51.50,23.96,38.71,67.13,48.10
1255,85.40,61.32,56.98,80.50,42.34,7.53,21.71,57.78,62.14,37.14,...,49.06,114.59,22.64,16.19,21.98,51.54,23.71,37.96,66.41,46.32
1256,85.09,61.90,57.23,79.98,42.93,7.32,21.66,57.87,62.49,37.49,...,49.05,115.71,22.72,16.34,21.92,50.02,23.45,38.54,66.45,46.14


In [32]:
raw_entry[0][0][2]

array(['"Industrials"'], dtype='<U13')

In [33]:
import pandas as pd
import numpy as np

# 假設 dfs 是你剛剛的那個字典
stock_raw = dfs['stock']

# Step 1: 解開 stock 的結構
stock_list = []

for col in stock_raw.columns:
    raw_entry = stock_raw[col].values[0]
    # raw_entry 長這樣 [[[['"MMM"'], ['"3M Co"'], ['"Industrials"']]]]
    # 需要連續提取兩層
    code = raw_entry[0][0][0][0]
    name = raw_entry[0][0][1][0]
    sector = raw_entry[0][0][2][0]
    # 去掉多餘的引號
    code = code.strip('"')
    name = name.strip('"')
    sector = sector.strip('"')
    stock_list.append({'code': code, 'name': name, 'sector': sector})

# Step 2: 轉成乾淨的 DataFrame
df_stock = pd.DataFrame(stock_list)

print(df_stock.head())

   code                           name                  sector
0   MMM                          3M Co             Industrials
1   ACE                    ACE Limited              Financials
2   ABT            Abbott Laboratories             Health Care
3   ANF  Abercrombie & Fitch Company A  Consumer Discretionary
4  ADBE              Adobe Systems Inc  Information Technology


In [34]:
df_stock

,code,name,sector
0,MMM,3M Co,Industrials
1,ACE,ACE Limited,Financials
2,ABT,Abbott Laboratories,Health Care
3,ANF,Abercrombie & Fitch Company A,Consumer Discretionary
4,ADBE,Adobe Systems Inc,Information Technology
...,...,...,...
447,XL,XL Capital,Financials
448,YHOO,Yahoo Inc.,Information Technology
449,YUM,Yum! Brands Inc,Consumer Discretionary
450,ZMH,Zimmer Holdings,Health Care


In [35]:
# Step 1: 原始收盤價矩陣
df_X = dfs['X']

# Step 2: 修改列名成股票代碼
df_X.columns = df_stock['code']

# Step 3: 行索引改成日期？（可選）
# 如果你有日期資料，可以補上日期作為index
# 假設你知道第一天是2000年1月1日，可以這樣做（但這個需要根據你的資料決定）
# date_range = pd.bdate_range(start='2000-01-01', periods=1258)
# df_X.index = date_range

print(df_X.head())

code     MMM    ACE    ABT    ANF   ADBE   AMD   AES    AET    AFL      A  \
0     126.75  30.51  40.05  21.75  25.60  7.01  3.24  42.04  31.59  19.14   
1     126.27  30.69  40.44  21.34  26.30  6.94  3.45  42.33  31.60  19.05   
2     127.19  31.97  40.70  22.11  27.51  7.16  3.45  42.75  32.00  19.96   
3     127.07  31.78  38.87  22.07  28.51  7.17  3.37  42.18  31.73  19.78   
4     125.70  31.17  39.85  22.40  27.13  6.69  3.35  42.23  31.80  18.82   

code  ...    WEC   WYNN    XEL   XRX   XLNX     XL   YHOO    YUM    ZMH   ZION  
0     ...  25.65  13.50  11.35  8.16  22.03  80.23  17.60  25.12  40.30  40.64  
1     ...  25.59  13.40  11.54  8.28  22.94  80.70  18.10  24.55  40.40  40.27  
2     ...  26.48  13.29  12.25  8.44  24.24  82.97  18.94  24.99  40.66  41.39  
3     ...  26.05  13.31  11.80  8.31  25.45  82.51  19.15  25.15  40.26  41.59  
4     ...  26.11  13.02  12.60  8.27  24.71  81.47  18.75  25.27  40.00  42.03  

[5 rows x 452 columns]


In [36]:
df_X

code,MMM,ACE,ABT,ANF,ADBE,AMD,AES,AET,AFL,A,...,WEC,WYNN,XEL,XRX,XLNX,XL,YHOO,YUM,ZMH,ZION
0,126.75,30.51,40.05,21.75,25.60,7.01,3.24,42.04,31.59,19.14,...,25.65,13.50,11.35,8.16,22.03,80.23,17.60,25.12,40.30,40.64
1,126.27,30.69,40.44,21.34,26.30,6.94,3.45,42.33,31.60,19.05,...,25.59,13.40,11.54,8.28,22.94,80.70,18.10,24.55,40.40,40.27
2,127.19,31.97,40.70,22.11,27.51,7.16,3.45,42.75,32.00,19.96,...,26.48,13.29,12.25,8.44,24.24,82.97,18.94,24.99,40.66,41.39
3,127.07,31.78,38.87,22.07,28.51,7.17,3.37,42.18,31.73,19.78,...,26.05,13.31,11.80,8.31,25.45,82.51,19.15,25.15,40.26,41.59
4,125.70,31.17,39.85,22.40,27.13,6.69,3.35,42.23,31.80,18.82,...,26.11,13.02,12.60,8.27,24.71,81.47,18.75,25.27,40.00,42.03
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1253,86.26,63.15,58.35,83.85,42.84,7.77,22.09,58.40,62.52,36.98,...,49.67,113.28,22.99,16.38,22.44,52.58,24.05,38.66,67.42,48.83
1254,86.07,62.86,57.88,82.54,43.12,7.77,22.13,58.82,62.77,37.47,...,49.13,117.67,22.89,16.49,22.50,51.50,23.96,38.71,67.13,48.10
1255,85.40,61.32,56.98,80.50,42.34,7.53,21.71,57.78,62.14,37.14,...,49.06,114.59,22.64,16.19,21.98,51.54,23.71,37.96,66.41,46.32
1256,85.09,61.90,57.23,79.98,42.93,7.32,21.66,57.87,62.49,37.49,...,49.05,115.71,22.72,16.34,21.92,50.02,23.45,38.54,66.45,46.14


In [37]:
df_X.to_csv('stock_data.csv', index=False)
df_stock.to_csv('stock_info.csv', index=False)

In [40]:
pd.read_csv("stock_data.csv")

,MMM,ACE,ABT,ANF,ADBE,AMD,AES,AET,AFL,A,...,WEC,WYNN,XEL,XRX,XLNX,XL,YHOO,YUM,ZMH,ZION
0,126.75,30.51,40.05,21.75,25.60,7.01,3.24,42.04,31.59,19.14,...,25.65,13.50,11.35,8.16,22.03,80.23,17.60,25.12,40.30,40.64
1,126.27,30.69,40.44,21.34,26.30,6.94,3.45,42.33,31.60,19.05,...,25.59,13.40,11.54,8.28,22.94,80.70,18.10,24.55,40.40,40.27
2,127.19,31.97,40.70,22.11,27.51,7.16,3.45,42.75,32.00,19.96,...,26.48,13.29,12.25,8.44,24.24,82.97,18.94,24.99,40.66,41.39
3,127.07,31.78,38.87,22.07,28.51,7.17,3.37,42.18,31.73,19.78,...,26.05,13.31,11.80,8.31,25.45,82.51,19.15,25.15,40.26,41.59
4,125.70,31.17,39.85,22.40,27.13,6.69,3.35,42.23,31.80,18.82,...,26.11,13.02,12.60,8.27,24.71,81.47,18.75,25.27,40.00,42.03
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1253,86.26,63.15,58.35,83.85,42.84,7.77,22.09,58.40,62.52,36.98,...,49.67,113.28,22.99,16.38,22.44,52.58,24.05,38.66,67.42,48.83
1254,86.07,62.86,57.88,82.54,43.12,7.77,22.13,58.82,62.77,37.47,...,49.13,117.67,22.89,16.49,22.50,51.50,23.96,38.71,67.13,48.10
1255,85.40,61.32,56.98,80.50,42.34,7.53,21.71,57.78,62.14,37.14,...,49.06,114.59,22.64,16.19,21.98,51.54,23.71,37.96,66.41,46.32
1256,85.09,61.90,57.23,79.98,42.93,7.32,21.66,57.87,62.49,37.49,...,49.05,115.71,22.72,16.34,21.92,50.02,23.45,38.54,66.45,46.14


In [39]:
pd.read_csv('stock_info.csv')

,code,name,sector
0,MMM,3M Co,Industrials
1,ACE,ACE Limited,Financials
2,ABT,Abbott Laboratories,Health Care
3,ANF,Abercrombie & Fitch Company A,Consumer Discretionary
4,ADBE,Adobe Systems Inc,Information Technology
...,...,...,...
447,XL,XL Capital,Financials
448,YHOO,Yahoo Inc.,Information Technology
449,YUM,Yum! Brands Inc,Consumer Discretionary
450,ZMH,Zimmer Holdings,Health Care
